In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz
!rm spark-3.4.0-bin-hadoop3.tgz   # Tidying up

--2023-05-16 04:10:51--  https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388407094 (370M) [application/x-gzip]
Saving to: ‘spark-3.4.0-bin-hadoop3.tgz’

spark-3.4.0-bin-had 100%[===================>] 370.41M   268MB/s    in 1.4s    

2023-05-16 04:10:52 (268 MB/s) - ‘spark-3.4.0-bin-hadoop3.tgz’ saved [388407094/388407094]



In [ ]:
# Setting up our environmental variables: 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

In [ ]:
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/BIGDATA_assignment_2/

/content/drive/MyDrive/BIGDATA_assignment_2


In [ ]:
!ls

article-ids.txt		    spark-3.3.1-bin-hadoop3
combiner_bigram.py	    spark-3.4.0-bin-hadoop3
COMP47470_assignment_2.pdf  spark-3.4.0-bin-hadoop3.tgz.1
lab4_config.sh		    spark-3.4.0-bin-hadoop3.tgz.2
mapper_bigram.py	    stopwords.txt
mapper_co-occur.py	    tester_args_mapper.py
nohup.out		    tester_args_reduce.py
parse_article.py	    tester_hadoop.py
parse_article.sh	    wikipedia-ml
reducer_bigram.py	    wikipedia-ml-raw
reducer_co-occur.py


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

In [ ]:
import multiprocessing
print(multiprocessing.cpu_count())

2


In [ ]:
text_files = spark.sparkContext.wholeTextFiles("./wikipedia-ml/*.txt")
#test
print("Number of Lines total: ", text_files.count())


Number of Lines total:  42


In [ ]:
#test
count_word = text_files.filter(lambda x: "Machine" in x[1]).count()

print(f"Number of lines with 'Machine': {count_word}")

Number of lines with 'Machine': 42


In [ ]:
# Extract the file names from the keys and print them
file_names = text_files.keys().collect()
for name in file_names:
    print(name)

file:/content/drive/MyDrive/BIGDATA_assignment_2/wikipedia-ml/article_Machine learning - Wikipedia_2013_1_2.txt
file:/content/drive/MyDrive/BIGDATA_assignment_2/wikipedia-ml/article_Machine learning - Wikipedia_2013_6_27.txt
file:/content/drive/MyDrive/BIGDATA_assignment_2/wikipedia-ml/article_Machine learning - Wikipedia_2014_1_4.txt
file:/content/drive/MyDrive/BIGDATA_assignment_2/wikipedia-ml/article_Machine learning - Wikipedia_2014_6_4.txt
file:/content/drive/MyDrive/BIGDATA_assignment_2/wikipedia-ml/article_Machine learning - Wikipedia_2015_1_4.txt
file:/content/drive/MyDrive/BIGDATA_assignment_2/wikipedia-ml/article_Machine learning - Wikipedia_2015_6_30.txt
file:/content/drive/MyDrive/BIGDATA_assignment_2/wikipedia-ml/article_Machine learning - Wikipedia_2016_1_1.txt
file:/content/drive/MyDrive/BIGDATA_assignment_2/wikipedia-ml/article_Machine learning - Wikipedia_2016_6_30.txt
file:/content/drive/MyDrive/BIGDATA_assignment_2/wikipedia-ml/article_Machine learning - Wikipedia_20

In [ ]:
#stopwords = spark.sparkContext.textFile("./stopwords.txt")

#print("Number of words total: ", stopwords.count())

In [ ]:
first_file = text_files.first()
print(first_file[1][:100])

This is an old revision of this page, as edited by 86.92.21.59 talk at 19:55, 2 January 2013. The pr


In [ ]:
few_files = text_files.take(2)
for file in few_files:
    print("File: ", file[0])
    print("Content: ", file[1][:700])

File:  file:/content/drive/MyDrive/BIGDATA_assignment_2/wikipedia-ml/article_Machine learning - Wikipedia_2013_1_2.txt
Content:  This is an old revision of this page, as edited by 86.92.21.59 talk at 19:55, 2 January 2013. The present address URL is a permanent link to this revision, which may differ significantly from the current revision.
Machine learning, a branch of artificial intelligence, is about the construction and study of systems that can learn from data. For example, a machine learning system could be trained on email messages to learn to distinguish between spam and non-spam messages. After learning, it can then be used to classify new email messages into spam and non-spam folders. 
The core of machine learning deals with representation and generalization. Representation of data instances and functions ev
File:  file:/content/drive/MyDrive/BIGDATA_assignment_2/wikipedia-ml/article_Machine learning - Wikipedia_2013_6_27.txt
Content:  This is an old revision of this page, as

In [ ]:
with open("./stopwords.txt", 'r') as f:
    stopwords = f.read().split()

print(stopwords[:10])


['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and']


In [ ]:
import itertools
import string

def rem_punc(arr):
     arr = arr.translate(str.maketrans('', '', string.punctuation))
     return arr

def rem_sw(arr, stopwords):
     new_arr=[]
     for i in arr:
          if i in stopwords:
               pass
          else:
               new_arr.append(i)
     return new_arr


def map_cooccur(group, stopwords):
    for g in group:
        rg = rem_punc(g[1])
        words = rg.lower().split()
        words = rem_sw(words, stopwords)
        pairs = []
        for i in range(0, len(words)):
            for j in range(i+1, len(words)):
                pairs.append((words[i], words[j])) # tuple of words and count
    # returned as key value pair with count of 1 for each
    res = [(pair, 1) for pair in pairs]
    return res


In [ ]:
def reducer(count1, count2):
    return count1 + count2

In [ ]:
# chat gpt function
import re

def extract_year(filename):
    match = re.search(r'_(\d{4})_', filename)
    if match:
        return match.group(1)
    else:
        return None

years = text_files.groupBy(lambda x: extract_year(x[0]))

In [ ]:
groups = years.collect()
print(groups)

[('2013', <pyspark.resultiterable.ResultIterable object at 0x7f7778acd510>), ('2014', <pyspark.resultiterable.ResultIterable object at 0x7f7778acf5b0>), ('2015', <pyspark.resultiterable.ResultIterable object at 0x7f7778ace740>), ('2016', <pyspark.resultiterable.ResultIterable object at 0x7f7778acf280>), ('2017', <pyspark.resultiterable.ResultIterable object at 0x7f7778acf9d0>), ('2019', <pyspark.resultiterable.ResultIterable object at 0x7f7778acf0a0>), ('2020', <pyspark.resultiterable.ResultIterable object at 0x7f7778acf460>), ('2018', <pyspark.resultiterable.ResultIterable object at 0x7f777066d1e0>), ('2021', <pyspark.resultiterable.ResultIterable object at 0x7f777066c460>), ('2022', <pyspark.resultiterable.ResultIterable object at 0x7f777066dd80>), ('2023', <pyspark.resultiterable.ResultIterable object at 0x7f777066e530>)]


In [ ]:
years

PythonRDD[11] at collect at <ipython-input-31-86814a6d380a>:1

In [ ]:
#from functools import reduce

group_results = years.map(lambda x: (x[0], map_cooccur(x[1], stopwords))) #.reduceByKey(lambda x, y: reduce(lambda a, b: a + b, [x, y]))

flat_results = group_results.values().flatMap(lambda x: x).reduceByKey(lambda x, y: x + y)

# could not find a way to run the reduce properly and keep the year data so chose to move ahead with just the one RDD
#final_results = flat_results.map(lambda x: ((extract_year(x[0]), x[0][0], x[0][1]), x[1])).groupByKey() 

In [ ]:
group_results

PythonRDD[16] at RDD at PythonRDD.scala:53

In [ ]:
flat_results.take(10)

[(('wernick', 'yourganov'), 6),
 (('wernick', 'machine'), 77),
 (('wernick', 'learning'), 103),
 (('wernick', 'medical'), 6),
 (('wernick', '4'), 14),
 (('wernick', 'july'), 7),
 (('wernick', '2010'), 11),
 (('wernick', '2538'), 6),
 (('wernick', 'phil'), 3),
 (('wernick', 'simon'), 3)]

In [ ]:
from pyspark.sql.functions import col, split

schema_df = flat_results.map(lambda x: (x[0][0], x[0][1], x[1])).toDF(["word_0", "word_1", "count"])

# sorting in advance
schema_df = schema_df.orderBy(col("count").desc()) 


In [ ]:
schema_df.select("*").show(10)

+---------+---------+-----+
|   word_0|   word_1|count|
+---------+---------+-----+
| learning|retrieved| 9436|
| learning| learning| 9381|
| learning|  machine| 7444|
|  machine| learning| 7199|
|  machine|retrieved| 6836|
|  machine|  machine| 5449|
|retrieved|retrieved| 4575|
|     isbn| learning| 4183|
|     isbn|retrieved| 3816|
|retrieved| learning| 3594|
+---------+---------+-----+
only showing top 10 rows



## Part2

In [ ]:
import string 

import itertools
import string

def rem_punc(arr):
     arr = arr.translate(str.maketrans('', '', string.punctuation))
     return arr

def map_bigram(group):
    for g in group:
        rg = rem_punc(g[1])
        rg = rg.strip()
        words = rg.lower().split()
        pairs = []
        for i in range(2, len(words)):
            pairs.append((words[i-2], words[i-1], words[i]))
        res = [(pair, 1) for pair in pairs]
        return res

In [ ]:
group_results = years.map(lambda x: (x[0], map_bigram(x[1]))) #.reduceByKey(lambda x, y: reduce(lambda a, b: a + b, [x, y]))

flat_results = group_results.values().flatMap(lambda x: x).reduceByKey(lambda x, y: x + y)

In [ ]:
flat_results.take(10)

[(('is', 'an', 'old'), 11),
 (('an', 'old', 'revision'), 11),
 (('old', 'revision', 'of'), 11),
 (('as', 'edited', 'by'), 11),
 (('edited', 'by', '86922159'), 1),
 (('86922159', 'talk', 'at'), 1),
 (('2', 'january', '2013'), 1),
 (('january', '2013', 'the'), 1),
 (('the', 'present', 'address'), 11),
 (('address', 'url', 'is'), 11)]

In [ ]:
schema_df2 = flat_results.map(lambda x: (x[0][0], x[0][1], x[0][2], x[1])).toDF(["word_0", "word_1", "word_1", "count"])

# sorting in advance
schema_df2 = schema_df2.orderBy(col("count").desc()) 

In [ ]:
schema_df2.select("*").show(10)

+-------+----------+----------+-----+
| word_0|    word_1|    word_1|count|
+-------+----------+----------+-----+
|     of|   machine|  learning|  104|
|      a|       set|        of|  102|
|machine|  learning|algorithms|   65|
|machine|  learning|        is|   61|
| sparse|dictionary|  learning|   42|
|     of|         a|       set|   38|
|     be|      used|        to|   37|
|      a|   machine|  learning|   37|
|     in|   machine|  learning|   36|
|    the|     field|        of|   35|
+-------+----------+----------+-----+
only showing top 10 rows

